# LLVM Python libraries

In [1]:
%run ~/.jupyter/config.ipy

In [100]:
import llvmlite.binding as llvm
import llvmlite.ir as ir
from ctypes import CFUNCTYPE, c_double, c_int

# Compile from LLVM code

From [here](http://llvmlite.pydata.org/en/latest/user-guide/binding/examples.html).

We want to compile the following llvm.

In [32]:
llvm_ir = """
   define double @fpadd(double %".1", double %".2") {
   entry:
     %res = fadd double %".1", %".2"
     ret double %res
   }
"""

First we need to initialize some things.

In [33]:
llvm.initialize()
llvm.initialize_native_target()
llvm.initialize_native_asmprinter()

Now setup a compiling engine for this machine?

In [34]:
target = llvm.Target.from_default_triple()
target_machine = target.create_target_machine()
# And an execution engine with an empty backing module
backing_mod = llvm.parse_assembly("")
engine = llvm.create_mcjit_compiler(backing_mod, target_machine)

And parse the code

In [35]:
mod = llvm.parse_assembly(llvm_ir)
mod.verify()
engine.add_module(mod)
engine.finalize_object()
engine.run_static_constructors()

In [36]:
func_ptr = engine.get_function_address("fpadd")
print(func_ptr)

140560887226368


In [37]:
cfunc = CFUNCTYPE(c_double, c_double, c_double)(func_ptr)
res = cfunc(1.0, 3.5)
print("fpadd(...) =", res)

fpadd(...) = 4.5


# Compile by building block

From [here](http://llvmlite.pydata.org/en/latest/user-guide/ir/examples.html).

In [104]:
func_type = ir.FunctionType(                    
    ir.IntType(32), (ir.IntType(32), ir.IntType(32)) 
)                                                    

module = ir.Module("module")
func = ir.Function(
    module, func_type, "f_name"
)

block = func.append_basic_block("entry")
builder = ir.IRBuilder(block)
a, b = func.args

result = builder.add(a, b, name="res")
builder.ret(result)
print(func)
print("--------------")
print(module)

define i32 @"f_name"(i32 %".1", i32 %".2") 
{
entry:
  %"res" = add i32 %".1", %".2"
  ret i32 %"res"
}

--------------
; ModuleID = "module"
target triple = "unknown-unknown-unknown"
target datalayout = ""

define i32 @"f_name"(i32 %".1", i32 %".2") 
{
entry:
  %"res" = add i32 %".1", %".2"
  ret i32 %"res"
}



## Compile and run

In [77]:
llvm.initialize()
llvm.initialize_native_target()
llvm.initialize_native_asmprinter()

In [78]:
target = llvm.Target.from_default_triple()
target_machine = target.create_target_machine()
# And an execution engine with an empty backing module
backing_mod = llvm.parse_assembly("")
engine = llvm.create_mcjit_compiler(backing_mod, target_machine)

In [96]:
mod = llvm.parse_assembly(str(module))

engine.add_module(mod)
engine.finalize_object()
engine.run_static_constructors()

In [98]:
func_ptr = engine.get_function_address("f_name")
print(func_ptr)

140560790777856


In [103]:
cfunc = CFUNCTYPE(c_int, c_int, c_int)(func_ptr)
res = cfunc(1, 4)
print("fpadd(...) =", res)

fpadd(...) = 5
